In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

import csv
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.metrics import confusion_matrix
import glob

In [22]:
file_dir = "/content/drive/MyDrive/APS360 Project/"
test_path = file_dir + "vegetable dataset/test/"

# 20 vegetable classes
vegetable_classes = ['asparagus', 'bell pepper', 'broccoli', 'cabbage', 'carrot', 'celery', 'chilli pepper', 'corn', 'cucumber', 'eggplant', 
                     'lettuce', 'mushroom', 'onion', 'peas', 'potato', 'pumpkin', 'raddish', 'spinach', 'sweet potato', 'tomato']

In [4]:
def get_accuracy(model, data):
    correct = 0
    total = 0
    for imgs, labels in data:
        
        
        #############################################
        #To Enable GPU Usage
        if use_cuda and torch.cuda.is_available():
          imgs = imgs.cuda()
          labels = labels.cuda()
        #############################################
        
        output = model(imgs)
        
        #select index with maximum prediction score
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]
    return correct / total

In [5]:
class VeggieCNN(nn.Module):
    def __init__(self):
        super(VeggieCNN, self).__init__()
        self.name = "VeggieCNN"
        self.conv1 = nn.Conv2d(256, 256, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(256 * 1 * 1, 20)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, 256 * 1 * 1)
        x = self.fc1(x)
        return x

In [27]:
## load model
model_path =  file_dir + "saved_model"
model = VeggieCNN().cuda()
state = torch.load(model_path)
model.load_state_dict(state)

<All keys matched successfully>

In [57]:
np.random.seed(1000)

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                                transforms.Resize((224, 224))])
test_set = torchvision.datasets.ImageFolder(test_path, transform=transform)

In [58]:
test_loader = DataLoader(test_set, batch_size = 1, shuffle=True)

In [9]:
import torchvision.models
alexnet = torchvision.models.alexnet(pretrained=True)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 127MB/s]


In [59]:
import os
FEATURE_PATH = '/content/drive/MyDrive/APS360 Project/features/'

valid_labels = ['cucumber']

def alexnet_features(data_loader, path):
  n=0
  for img, label in iter(data_loader):

    if vegetable_classes[label] in valid_labels:
      features = alexnet.features(img)
      features = torch.from_numpy(features.detach().numpy())

      dir_path = path + str(vegetable_classes[label])
      try:
        os.makedirs(dir_path)
      except FileExistsError:
        pass
      torch.save(features.squeeze(0), dir_path + "/" + str(n) + ".tensor")
      n+=1

In [61]:
alexnet_features(test_loader, FEATURE_PATH + "test/")

In [42]:
test_feature_path = FEATURE_PATH + "test/"

In [66]:
alex_test_set = torchvision.datasets.DatasetFolder(FEATURE_PATH + "test/", loader=torch.load, extensions=('.tensor'))
alex_test_loader = DataLoader(alex_test_set, batch_size = 34, shuffle=True, drop_last=True)

In [67]:
use_cuda = True
test_acc =  get_accuracy(model, alex_test_loader)
print("Final Test Accuracy: ", test_acc)

Final Test Accuracy:  0.7455882352941177


In [65]:
for i, label in enumerate(vegetable_classes):
  veggie_path = "/content/drive/MyDrive/alex_test/" + label
  features = torchvision.datasets.DatasetFolder(veggie_path, loader=torch.load, extensions=('.tensor'))
  test_loader = torch.utils.data.DataLoader(features, batch_size=1, shuffle=True)

  correct = 0
  false_positive = [0]*20
  for imgs, labels in test_loader:
        #############################################
        #To Enable GPU Usage
        if torch.cuda.is_available():
          imgs = imgs.cuda()
          labels = labels.cuda()
        #############################################
        
        
        output = model(imgs)
        probs = F.softmax(output, dim=1)
        _, predicted = torch.max(probs, 1)
        # print(predicted.item())

        if predicted.item() == i:
          correct += 1
        else:
          false_positive[predicted.item()] += 1

  print(vegetable_classes[i] + ":")
  print(f"\tAccuracy: {correct/len(test_loader)*100:.2f}%")
  print(f"\tMost frequent false postive: {vegetable_classes[np.argmax(false_positive)]}")

asparagus:
	Accuracy: 87.50%
	Most frequent false postive: celery
bell pepper:
	Accuracy: 58.33%
	Most frequent false postive: tomato
broccoli:
	Accuracy: 100.00%
	Most frequent false postive: asparagus
cabbage:
	Accuracy: 71.43%
	Most frequent false postive: potato
carrot:
	Accuracy: 91.43%
	Most frequent false postive: sweet potato
celery:
	Accuracy: 14.29%
	Most frequent false postive: asparagus
chilli pepper:
	Accuracy: 65.62%
	Most frequent false postive: spinach
corn:
	Accuracy: 80.56%
	Most frequent false postive: asparagus
cucumber:
	Accuracy: 51.52%
	Most frequent false postive: asparagus
eggplant:
	Accuracy: 80.56%
	Most frequent false postive: carrot
lettuce:
	Accuracy: 52.50%
	Most frequent false postive: cabbage
mushroom:
	Accuracy: 51.52%
	Most frequent false postive: celery
onion:
	Accuracy: 94.29%
	Most frequent false postive: mushroom
peas:
	Accuracy: 93.94%
	Most frequent false postive: celery
potato:
	Accuracy: 100.00%
	Most frequent false postive: asparagus
pumpkin: